In [3]:
import os
import sys
import torch
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

from torch.distributions import Normal, MultivariateNormal
from torch.distributions import Bernoulli, Beta, Poisson
from bdp.models.random_fields.poisson_covariance import MertonBirthPoissonCovariance

%matplotlib inline

In [4]:
model_param = MertonBirthPoissonCovariance.get_parameters()
inference_param = MertonBirthPoissonCovariance.get_inference_parameters()

model_param.update({"birth_intensity":19})
model_param.update({"number_of_realizations":5})

inference_param.update({"nmc": 3000,
                        "burning": 1500,
                        "metrics_logs": 50})

merton_birth = MertonBirthPoissonCovariance(None, None, None, **model_param)

New Model Set For Inference


In [5]:
model_param

{'locations_dimension': 2,
 'jump_size_scale_prior': 1.0,
 'birth_intensity': 19,
 'returns_mean_a': 1.0,
 'returns_mean_b': 1.0,
 'prior_locations_mean': 0.0,
 'prior_locations_std': 1.0,
 'prior_sigma_mean': 0.0,
 'prior_sigma_std': 1.0,
 'prior_length_mean': 0.0,
 'prior_length_std': 1.0,
 'kernel_parameters': {'kernel_sigma': 0.5,
  'kernel_lenght_scales': [1.0, 2.0]},
 'number_of_processes': 4,
 'number_of_realizations': 5,
 'model_path': 'c:\\users\\cesar\\desktop\\projects\\general\\deep_random_fields\\results'}

In [27]:
number_of_realizations = merton_birth.number_of_realizations

#Births
birth_distribution = Poisson(merton_birth.birth_intensity)
birth_numbers = birth_distribution.sample((number_of_realizations,)).long()
assets_in_the_market = birth_numbers.cumsum(dim=0)
total_assets_in_history = assets_in_the_market[-1]

#Locations
locations_history = merton_birth.locations_prior.sample(sample_shape=(total_assets_in_history,))

#Kernel
kernel, kernel_eval = merton_birth.define_kernel(merton_birth.kernel_sigma, merton_birth.kernel_lenght_scales)
covariance_diffusion = kernel_eval(locations_history)

# expected returns -------------
returns_mean_prior = torch.ones(total_assets_in_history) * merton_birth.returns_mean_a
returns_covariance_prior = merton_birth.returns_mean_b*covariance_diffusion
expected_returns_distribution = MultivariateNormal(returns_mean_prior,returns_covariance_prior)
expected_returns_history = expected_returns_distribution.sample()

In [32]:
#MARKET BIRTH PROCESS
for time_index in range(merton_birth.number_of_realizations):
    current_number_of_assets = assets_in_the_market[time_index]
    current_expected_returns = expected_returns_history[:current_number_of_assets]
    current_covariance = covariance_diffusion[:current_number_of_assets,:current_number_of_assets]
    current_log_returns_distribution = MultivariateNormal(current_expected_returns,current_covariance)
    current_log_returns = current_log_returns_distribution.sample()
    
    log_returns[time_index,:current_number_of_assets] = current_log_returns 

In [ ]:
# Jumps -------------------------

# sample from intensity prior
arrivals_intensity = self.arrivals_intensity_prior.sample()
self.arrivals_indicator_prior = Bernoulli(arrivals_intensity)
arrivals_indicator = self.arrivals_indicator_prior.sample(sample_shape=(self.number_of_realizations,))

# diffusion -------------

# covariance -------------------
locations = self.locations_prior.sample(sample_shape=(self.number_of_processes,))



# are we missing time scale Delta t? Assumed == 1
diffusive_log_returns = expected_return[None, :] + \
                        MultivariateNormal(torch.zeros(covariance_diffusion.shape[0]), covariance_diffusion).sample(
                            sample_shape=(self.number_of_realizations,))

log_returns = diffusive_log_returns + jumps_size * arrivals_indicator[:, None]
